In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [26]:
climate = pd.read_csv('Climate_Zone_of_each_country.csv')
population = pd.read_csv('FAOSTAT_data_Annual_Population_2009-2018.csv')
food_balance = pd.read_csv('FAOSTAT_data_Food_Balance_2009-2013.csv')
food_balance2 = pd.read_csv('FAOSTAT_data_Food_Balance_2014-2018.csv')
fertilizer = pd.read_csv('FAOSTAT_Fertilizer.csv')
pesticides = pd.read_csv('FAOSTAT_Pesticides.csv')

In [27]:
crops = pd.read_csv('Production_Crops_E_All_Data_NOFLAG.csv', encoding='ISO-8859-1')

# Question 1 

In [28]:
frames = [food_balance,food_balance2]
food_balance=pd.concat(frames)
food_balance.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBSH,Food Balances (old methodology and population),2,Afghanistan,5142,Food,2511,Wheat and products,2009,2009,1000 tonnes,4538.0,S,Standardized data
1,FBSH,Food Balances (old methodology and population),2,Afghanistan,5142,Food,2511,Wheat and products,2010,2010,1000 tonnes,4605.0,S,Standardized data
2,FBSH,Food Balances (old methodology and population),2,Afghanistan,5142,Food,2511,Wheat and products,2011,2011,1000 tonnes,4711.0,S,Standardized data
3,FBSH,Food Balances (old methodology and population),2,Afghanistan,5142,Food,2511,Wheat and products,2012,2012,1000 tonnes,4810.0,S,Standardized data
4,FBSH,Food Balances (old methodology and population),2,Afghanistan,5142,Food,2511,Wheat and products,2013,2013,1000 tonnes,4895.0,S,Standardized data


In [29]:
food_balance.iloc[food_balance['Value'].argsort()[-3:]]

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
13153,FBS,New Food Balances,351,China,5142,Food,2605,"Vegetables, Other",2017,2017,1000 tonnes,450576.0,A,"Aggregate, may include official, semi-official..."
14491,FBS,New Food Balances,41,"China, mainland",5142,Food,2605,"Vegetables, Other",2018,2018,1000 tonnes,451702.0,Im,FAO data based on imputation methodology
13154,FBS,New Food Balances,351,China,5142,Food,2605,"Vegetables, Other",2018,2018,1000 tonnes,455116.0,A,"Aggregate, may include official, semi-official..."


In [71]:
highest_demand_crops = food_balance.groupby('Item')['Value'].agg('sum').nlargest(4)
highest_demand_crops

Item
Vegetables, Other          11769967.0
Milk - Excluding Butter     6272191.0
Rice and products           6125966.0
Wheat and products          5568366.0
Name: Value, dtype: float64

# Question 2

In [31]:
crops.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,ha,NaN,NaN,NaN,...,11029.0,11210.0,13469.0,13490.0,14114.0,13703.0,14676.0,19481.0,19793.0,20053.0
1,2,Afghanistan,221,"Almonds, with shell",5419,Yield,hg/ha,NaN,NaN,NaN,...,39154.0,49955.0,45000.0,45960.0,29910.0,19996.0,16521.0,16859.0,13788.0,17161.0
2,2,Afghanistan,221,"Almonds, with shell",5510,Production,tonnes,NaN,NaN,NaN,...,43183.0,56000.0,60611.0,62000.0,42215.0,27400.0,24246.0,32843.0,27291.0,34413.0
3,2,Afghanistan,711,"Anise, badian, fennel, coriander",5312,Area harvested,ha,NaN,NaN,NaN,...,17748.0,17000.0,19500.0,18500.0,18500.0,30000.0,25000.0,25638.0,27582.0,25785.0
4,2,Afghanistan,711,"Anise, badian, fennel, coriander",5419,Yield,hg/ha,NaN,NaN,NaN,...,6203.0,6000.0,6414.0,6757.0,6757.0,7167.0,7200.0,7037.0,6954.0,7036.0


In [32]:
#to know what kind of crops appeared in datasets
crops['Item'].unique()

array(['Almonds, with shell', 'Anise, badian, fennel, coriander',
       'Apples', 'Apricots', 'Barley', 'Berries nes', 'Figs',
       'Fruit, citrus nes', 'Fruit, fresh nes', 'Fruit, stone nes',
       'Grapes', 'Linseed', 'Maize', 'Melons, other (inc.cantaloupes)',
       'Millet', 'Nuts nes', 'Olives', 'Onions, dry', 'Oranges',
       'Peaches and nectarines', 'Pears', 'Pistachios', 'Plums and sloes',
       'Potatoes', 'Pulses nes', 'Rice, paddy',
       'Rice, paddy (rice milled equivalent)', 'Seed cotton',
       'Sesame seed', 'Spices nes', 'Sugar beet', 'Sugar cane',
       'Sunflower seed', 'Vegetables, fresh nes', 'Walnuts, with shell',
       'Watermelons', 'Wheat', 'Cereals, Total', 'Citrus Fruit, Total',
       'Fibre Crops Primary', 'Fruit Primary', 'Oilcrops',
       'Oilcrops, Cake Equivalent', 'Oilcrops, Oil Equivalent',
       'Pulses, Total', 'Roots and Tubers, Total', 'Sugar Crops Primary',
       'Treenuts, Total', 'Vegetables Primary', 'Beans, dry',
       'Beans,

In [33]:
crops = crops.fillna(0)



In [45]:
filter_column = list(crops)
#print(filter_column)
filter_column.remove('Area Code')
filter_column.remove('Area')
filter_column.remove('Item Code')
filter_column.remove('Item')
filter_column.remove('Element Code')
filter_column.remove('Element')
filter_column.remove('Unit')
#print(filter_column)

In [46]:
crops['sum_crops'] = crops[filter_column].sum(axis=1)

In [43]:
crops.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,sum_crops
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,ha,0.0,0.0,0.0,...,11210.0,13469.0,13490.0,14114.0,13703.0,14676.0,19481.0,19793.0,20053.0,373623.0
1,2,Afghanistan,221,"Almonds, with shell",5419,Yield,hg/ha,0.0,0.0,0.0,...,49955.0,45000.0,45960.0,29910.0,19996.0,16521.0,16859.0,13788.0,17161.0,883755.0
2,2,Afghanistan,221,"Almonds, with shell",5510,Production,tonnes,0.0,0.0,0.0,...,56000.0,60611.0,62000.0,42215.0,27400.0,24246.0,32843.0,27291.0,34413.0,816387.0
3,2,Afghanistan,711,"Anise, badian, fennel, coriander",5312,Area harvested,ha,0.0,0.0,0.0,...,17000.0,19500.0,18500.0,18500.0,30000.0,25000.0,25638.0,27582.0,25785.0,381519.0
4,2,Afghanistan,711,"Anise, badian, fennel, coriander",5419,Yield,hg/ha,0.0,0.0,0.0,...,6000.0,6414.0,6757.0,6757.0,7167.0,7200.0,7037.0,6954.0,7036.0,217429.0


In [77]:
#total
total_of_vege = crops.loc[crops['Item']=='Vegetables Primary', 'sum_crops'].sum()
print("total of vegetables is ", total_of_vege)
#average
num_of_vege_apperance = len(crops.loc[crops['Item']=='Vegetables Primary'])
print("number of appearance of vegetables : ",num_of_vege_apperance)
print("average of vegetables is ", total_of_vege/num_of_vege_apperance)
#median
median_of_vege = crops.loc[crops['Item']=='Vegetables Primary', 'sum_crops'].median()
print("median of vegetables is ", median_of_vege)

total of vegetables is  311075185194.0
number of appearance of vegetables :  746
average of vegetables is  416990864.8713137
median of vegetables is  11538206.0


In [73]:
#total
total_of_rice = crops.loc[crops['Item']=='Rice, paddy', 'sum_crops'].sum()
print("total of Rice, paddy is ", total_of_rice)
#average
num_of_rice_apperance = len(crops.loc[crops['Item']=='Rice, paddy'])
print("number of appearance of Rice, paddy : ",num_of_rice_apperance)
print("average of Rice, paddy is ", total_of_rice/num_of_rice_apperance)
#median
median_of_rice = crops.loc[crops['Item']=='Rice, paddy', 'sum_crops'].median()
print("median of Rice, paddy is ", median_of_rice)

total of Rice, paddy is  372683872562.0
number of appearance of Rice, paddy :  488
average of Rice, paddy is  763696460.1680328
median of Rice, paddy is  4421846.0


In [74]:
#total
total_of_Wheat = crops.loc[crops['Item']=='Wheat', 'sum_crops'].sum()
print("total of Wheat is ", total_of_Wheat)
#average
num_of_Wheat_apperance = len(crops.loc[crops['Item']=='Wheat'])
print("number of appearance of Wheat : ",num_of_Wheat_apperance)
print("average of Wheat is ", total_of_Wheat/num_of_Wheat_apperance)
#median
median_of_Wheat = crops.loc[crops['Item']=='Wheat', 'sum_crops'].median()
print("median of Wheat is ", median_of_Wheat)

total of Wheat is  387022025386.0
number of appearance of Wheat :  494
average of Wheat is  783445395.5182186
median of Wheat is  5372019.0


# Question 3

In [108]:
vege = crops.loc[(crops['Item']=='Vegetables Primary') & (crops['sum_crops']<median_of_vege),'Area']
#print(vege.unique())
print(len(vege),"out of total countries lower than median")

373 out of total countries lower than median


In [109]:
rice = crops.loc[(crops['Item']=='Rice, paddy') & (crops['sum_crops']<median_of_rice),'Area']
#print(rice.unique())
print(len(rice),"out of total countries lower than median")

244 out of total countries lower than median


In [110]:
wheat = crops.loc[(crops['Item']=='Wheat') & (crops['sum_crops']<median_of_Wheat),'Area']
#print(wheat.unique())
print(len(wheat),"out of total countries lower than median")

247 out of total countries lower than median


In [104]:
def intersection(lst1, lst2,lst3): 
    return list(set(lst1) & set(lst2) & set(lst3))

##print out countries that having three high semand crops lower than median
print(intersection(vege,rice,wheat)) 

['Turkmenistan', 'Kazakhstan', 'USSR', 'Burundi', 'Niger', 'Zimbabwe', 'Thailand', 'Small Island Developing States', 'Net Food Importing Developing Countries', 'Azerbaijan', 'Bulgaria', 'Angola', 'South-eastern Asia', 'Ukraine', 'Chad', 'Nepal', 'Chile', 'Rwanda', 'South Sudan', 'Bolivia (Plurinational State of)', 'Sudan', 'Tajikistan', 'Republic of Moldova', 'Nigeria', 'Ecuador', 'Melanesia', 'Guatemala', 'Honduras', 'Least Developed Countries', 'Bhutan', 'Russian Federation', 'Uganda', 'Bangladesh', 'Cameroon', 'Kenya', 'Syrian Arab Republic', 'Kyrgyzstan', 'Low Income Food Deficit Countries', 'Armenia', 'Africa', 'Iraq', 'Saudi Arabia', 'Afghanistan', 'Somalia', 'Middle Africa', 'Malawi', 'Albania', 'Mozambique', 'Mali', 'Zambia', 'Yugoslav SFR', 'Sudan (former)', 'Mauritania', 'Democratic Republic of the Congo', 'Portugal', 'Eswatini', 'Western Africa', 'Paraguay', 'Madagascar', 'United Republic of Tanzania', 'North Macedonia', 'Venezuela (Bolivarian Republic of)', 'Uzbekistan', 'E